In [1]:
!pip install faiss-cpu
!pip install transformers
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 9.7 MB/s eta 0:00:00:00:0100:01


In [2]:
import pandas as pd
import numpy as np
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import faiss

/home/mehmet/Downloads/PREFIX=/home/mehmetokpc/anaconda3/envs/logModel/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)    # Show all rows
pd.set_option('display.max_colwidth', None) # Allow wrapping for long values

In [9]:
model_path = "../modelAndDatabase/"
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [11]:
faiss_index_path = "../modelAndDatabase/faiss_index.index"
faiss_index = faiss.read_index(faiss_index_path)

In [15]:
import torch
import torch.nn as nn
class DimensionReducer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DimensionReducer, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)

# Model ve tokenizer ayarları
input_dim = 512  # Model çıkış boyutu
output_dim = 384  # FAISS indeks boyutu
dimension_reducer = DimensionReducer(input_dim, output_dim)

def encode_query(query):
    inputs = tokenizer(query, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model.encoder(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    reduced_embeddings = dimension_reducer(embeddings).detach().numpy()  # Boyut dönüşümünü uygulama
    return reduced_embeddings

def generate_answer(query, retrieved_info):
    input_text = f"Context: {retrieved_info}\nQuestion: {query}"
    inputs = tokenizer(input_text, return_tensors='pt', truncation=True, padding=True)
    outputs = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=100, num_beams=5, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def retrieve_and_generate(query):
    # Sorgu vektörünü hazırlama
    query_vector = encode_query(query)

    # En yakın komşuları bulma
    D, I = faiss_index.search(query_vector, k=5)

    # Bilgi çekme (örnek: İndekslerden bilgi parçalarını al)
    retrieved_info = " ".join([f"Information {i}" for i in I[0]])

    # Yanıt üretme
    answer = generate_answer(query, retrieved_info)
    return answer

In [16]:
query = "What was the status of the request from IP address 66.249.66.91 to /css/font/wyekan/font.woff on 2019-01-25 00:38:49?"
answer = retrieve_and_generate(query)
print("Answer:", answer)

Answer: The status of the request from IP address 66.249.66.91 to /css/font/wyekan/font.woff on 2019-01-25 00:38:49 was success.
